Создание черного одноканального изображения 500*600.

In [1]:
import numpy as np
from PIL import Image

data = np.zeros((500, 600), dtype='|u1')
img = Image.fromarray(data)
img.save('pictures/colors/black.png')

Создание белого одноканального изображения 500*600.

In [2]:
import numpy as np
from PIL import Image

data = np.full((500, 600), 255, dtype='|u1')
img = Image.fromarray(data)
img.save('pictures/colors/white.png')

Создание красного RGB изображения 500*600.
Используется собственная обертка в `main.py`.

In [3]:
from core import CustomImage
from names import RED

img = CustomImage(600, 500)
img.fill(RED)
img.save('pictures/colors/red.png')

Создание градиентного RGB изображения 500*600.

In [4]:
from core import CustomImage

width = 600
height = 500
img = CustomImage(width, height)
for i in range(width):
    for j in range(height):
        val = (i + j) % 256
        img.set((i, j), (val, val, val))
img.save('pictures/colors/gradient.png')

Построение простейшей прямой (1 алгоритм)

In [5]:
from core import CustomImage
from utils import float_range, star
from names import WHITE
from type import Color, ImageVertex


def line(v0: ImageVertex, v1: ImageVertex, image: CustomImage, color: Color):
    x0, y0 = v0
    x1, y1 = v1
    for t in float_range(0, 1, 0.01, 8):
        x = x0 * (1.0 - t) + x1 * t
        y = y0 * (1.0 - t) + y1 * t
        image.set((round(x), round(y)), color)


img = star(line, WHITE)
img.save('pictures/lines/line_1.png')

Второй вариант построения прямой (2 алгоритм)

In [6]:
from core import CustomImage
from utils import star
from names import WHITE
from type import Color


def line(v0: ImageVertex, v1: ImageVertex, image: CustomImage, color: Color):
    x0, y0 = v0
    x1, y1 = v1
    for x in range(x0, x1, 1):
        t = (x - x0) / float(x1 - x0)
        y = y0 * (1.0 - t) + y1 * t
        image.set((round(x), round(y)), color)


img = star(line, WHITE)
img.save('pictures/lines/line_2.png')

Третий вариант построения прямой (3 алгоритм)

In [7]:
from core import CustomImage
from utils import star
from names import WHITE
from type import Color


def line(v0: ImageVertex, v1: ImageVertex, image: CustomImage, color: Color):
    x0, y0 = v0
    x1, y1 = v1
    steep = False
    if abs(x0 - x1) < abs(y0 - y1):
        x0, y0 = y0, x0
        x1, y1 = y1, x1
        steep = True
    if x0 > x1:
        x0, x1 = x1, x0
        y0, y1 = y1, y0
    for x in range(x0, x1, 1):
        t = (x - x0) / float(x1 - x0)
        y = y0 * (1. - t) + y1 * t
        if steep:
            image.set((round(y), round(x)), color)
        else:
            image.set((round(x), round(y)), color)


img = star(line, WHITE)
img.save('pictures/lines/line_3.png')

Четвертый вариант построения прямой (Алгоритм Брезенхема)

In [8]:
from core import CustomImage
from utils import star
from names import WHITE


def line(v0: ImageVertex, v1: ImageVertex, image: CustomImage, color: Color):
    image.line(v0, v1, color)


img = star(line, WHITE)
img.save('pictures/lines/line_4.png')

Отрисовка вершин трехмерной модели.

In [9]:
from type import ObjVertex
from typing import Callable
from core import CustomImage, ObjModel
from names import WHITE, FOX, DEER

fox_obj = ObjModel(FOX)
deer_obj = ObjModel(DEER)

def draw_vertices(obj: ObjModel, _img: CustomImage, color: Color, transformation: Callable[[ObjVertex], ImageVertex]):
    for v in obj.vertices():
        _img.set(transformation(v), color)

transformations = [
    lambda v: (round(50 * v[0] + 500), round(50 * v[1] + 500)),
    lambda v: (round(100 * v[0] + 500), round(100 * v[1] + 500)),
    lambda v: (round(500 * v[0] + 500), round(500 * v[1] + 500)),
    lambda v: (round(4000 * v[0] + 500), round(4000 * v[1] + 500)),
]

for i, transform in enumerate(transformations):
    img = CustomImage(1000, 1000)
    draw_vertices(fox_obj, img, WHITE, transform)
    img.save(f'pictures/vertices/fox_{i + 1}.png')

for i, transform in enumerate(transformations):
    img = CustomImage(1000, 1000)
    draw_vertices(deer_obj, img, WHITE, transform)
    img.save(f'pictures/vertices/deer_{i + 1}.png')

Отрисовка ребер трехмерной модели.

In [10]:
from typing import Callable
from type import ObjVertex
from core import CustomImage, ObjModel
from names import WHITE, FOX, DEER

fox_obj = ObjModel(FOX)
deer_obj = ObjModel(DEER)

def draw_all_faces(obj: ObjModel, _img: CustomImage, color: Color, transformation: Callable[[ObjVertex], ImageVertex]):
    for v0, v1, v2 in obj.faces():
        v0 = transformation(v0)
        v1 = transformation(v1)
        v2 = transformation(v2)
        _img.line(v0, v1, color)
        _img.line(v1, v2, color)
        _img.line(v0, v2, color)

img = CustomImage(1000, 1000)
draw_all_faces(fox_obj, img, WHITE, lambda v: (4 * round(v[2]) + 500, 4 * -round(v[1]) + 500))
img.save('pictures/faces/fox_wire.png')

img = CustomImage(1500, 1500)
draw_all_faces(deer_obj, img, WHITE, lambda v: (round(v[0]) + 800, -round(v[1]) + 1500))
img.save('pictures/faces/deer_wire.png')